In [1]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.optimizers import RMSprop
from keras import initializers
from keras import regularizers
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.min_rows = 100
pd.options.display.max_columns = 100
# pd.options.display.min_columns = 100

Using TensorFlow backend.


In [2]:
batch_size = 271
original_dim = 327
latent_dim = 25
intermediate_dim = 100
epochs = 500
epsilon_std = 1
z_log_sigma_prior = np.log(0.7)

In [3]:
# encoder #
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
# z_log_sigma = Dense(latent_dim)(h)
## log_z_scale


z_log_sigma_input = Input(batch_shape = (batch_size, 1))
z_log_sigma = Dense(units = 1,  activation = "linear",
                    kernel_initializer=initializers.Ones(),
                    use_bias = False)(z_log_sigma_input)
# the weight of the prior #
# z_sigma_weight = Dense(units = 1, activation = "sigmoid",\
#                       kernel_regularizer = regularizers.l2(1e-4),\
#                       kernel_initializer = initializers.Zeros(),
#                       bias_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4),
#                       bias_initializer = initializers.Zeros())(z_sigma_input)

In [4]:
# sampling from latent space #
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
# z = Lambda(sampling)([z_mean, z_log_sigma])

In [5]:
# decoder #
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [6]:
# end-to-end autoencoder
vae = Model([x, z_log_sigma_input], x_decoded_mean)

# encoder, from inputs to latent space
encoder_z_mean = Model([x, z_log_sigma_input], z_mean)
encoder_z_log_sigma = Model([x, z_log_sigma_input], z_log_sigma)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [7]:
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean, label_smoothing=0.1)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

In [8]:
def recon_metric(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean, label_smoothing=0.1)
    return xent_loss

In [9]:
# vae.summary()

In [10]:
opt = optimizers.RMSprop(learning_rate=0.0005)
# opt = optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=opt, loss=vae_loss, metrics = [recon_metric])

In [11]:
movie = pd.read_csv('movie_analysis/movie.csv')
cast = movie.iloc[:,26:].to_numpy()
log_sigma_input = np.full((cast.shape[0], 1), z_log_sigma_prior)

In [12]:
import copy
test_manip = np.empty(shape=(327*2, 327))
idx = np.random.choice(a=list(range(2439)), size=327*2, replace=False)
for i in range(327):
    t1 = copy.deepcopy(cast[idx[2*i]])
    t1[i] = 1
    test_manip[(2*i)] = t1
    t0 = copy.deepcopy(cast[idx[2*i+1]])
    t0[i] = 0   
    test_manip[(2*i+1)] = t0
test_manip = test_manip[np.random.choice(a=list(range(327*2)), size=271*2, replace=False)]

In [13]:
from sklearn.model_selection import train_test_split
cast_train, test_obs = train_test_split(cast,train_size=1897,test_size = 542)
cast_test = np.concatenate((test_obs, test_manip), axis = 0)
log_sigma_input_train = np.full((cast_train.shape[0], 1), z_log_sigma_prior)
log_sigma_input_test = np.full((cast_test.shape[0], 1), z_log_sigma_prior)

In [14]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)

In [15]:
# fitting VAE #
# vae.fit([cast,log_sigma_input], cast,
#         shuffle=True,
#         epochs=epochs,
#         batch_size=batch_size)

# fitting VAE #
history = vae.fit([cast_train,log_sigma_input_train], cast_train,
        shuffle=True,
        epochs=epochs,
        verbose = 0,
        batch_size=batch_size,
        validation_data=([cast_test, log_sigma_input_test], cast_test),
        callbacks = [callback])

Train on 1897 samples, validate on 1084 samples
Epoch 1/500
1897/1897 [==============================] - 1s 458us/step - loss: 219.3754 - recon_metric: 219.3393 - val_loss: 209.2988 - val_recon_metric: 209.2589
Epoch 2/500
1897/1897 [==============================] - 0s 45us/step - loss: 202.7905 - recon_metric: 202.7480 - val_loss: 193.6397 - val_recon_metric: 193.5894
Epoch 3/500
1897/1897 [==============================] - 0s 44us/step - loss: 186.4964 - recon_metric: 186.4415 - val_loss: 175.6378 - val_recon_metric: 175.5700
Epoch 4/500
1897/1897 [==============================] - 0s 42us/step - loss: 167.5186 - recon_metric: 167.4443 - val_loss: 154.5607 - val_recon_metric: 154.4665
Epoch 5/500
1897/1897 [==============================] - 0s 41us/step - loss: 145.9914 - recon_metric: 145.8888 - val_loss: 132.1170 - val_recon_metric: 131.9864
Epoch 6/500
1897/1897 [==============================] - 0s 40us/step - loss: 124.6905 - recon_metric: 124.5517 - val_loss: 112.1178 - val_re

1897/1897 [==============================] - 0s 39us/step - loss: 72.3387 - recon_metric: 72.2459 - val_loss: 73.0520 - val_recon_metric: 72.9634
Epoch 104/500
1897/1897 [==============================] - 0s 39us/step - loss: 72.3348 - recon_metric: 72.2429 - val_loss: 73.0465 - val_recon_metric: 72.9586
Epoch 105/500
1897/1897 [==============================] - 0s 39us/step - loss: 72.3168 - recon_metric: 72.2256 - val_loss: 73.0382 - val_recon_metric: 72.9510
Epoch 106/500
1897/1897 [==============================] - 0s 39us/step - loss: 72.3215 - recon_metric: 72.2307 - val_loss: 73.0337 - val_recon_metric: 72.9471
Epoch 107/500
1897/1897 [==============================] - 0s 39us/step - loss: 72.3103 - recon_metric: 72.2196 - val_loss: 73.0290 - val_recon_metric: 72.9422
Epoch 108/500
1897/1897 [==============================] - 0s 39us/step - loss: 72.3093 - recon_metric: 72.2185 - val_loss: 73.0221 - val_recon_metric: 72.9360
Epoch 109/500
1897/1897 [=============================

1897/1897 [==============================] - 0s 40us/step - loss: 71.4441 - recon_metric: 71.1754 - val_loss: 72.3523 - val_recon_metric: 72.1099
Epoch 206/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4335 - recon_metric: 71.1623 - val_loss: 72.3496 - val_recon_metric: 72.1052
Epoch 207/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4265 - recon_metric: 71.1530 - val_loss: 72.3473 - val_recon_metric: 72.1012
Epoch 208/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4154 - recon_metric: 71.1397 - val_loss: 72.3369 - val_recon_metric: 72.0881
Epoch 209/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.4061 - recon_metric: 71.1280 - val_loss: 72.3312 - val_recon_metric: 72.0805
Epoch 210/500
1897/1897 [==============================] - 0s 39us/step - loss: 71.3963 - recon_metric: 71.1160 - val_loss: 72.3268 - val_recon_metric: 72.0743
Epoch 211/500
1897/1897 [=============================

1897/1897 [==============================] - 0s 40us/step - loss: 70.7275 - recon_metric: 70.2492 - val_loss: 71.8345 - val_recon_metric: 71.4061
Epoch 308/500
1897/1897 [==============================] - 0s 40us/step - loss: 70.7180 - recon_metric: 70.2374 - val_loss: 71.8309 - val_recon_metric: 71.4009
Epoch 309/500
1897/1897 [==============================] - 0s 39us/step - loss: 70.7148 - recon_metric: 70.2323 - val_loss: 71.8274 - val_recon_metric: 71.3957
Epoch 310/500
1897/1897 [==============================] - 0s 39us/step - loss: 70.7050 - recon_metric: 70.2206 - val_loss: 71.8231 - val_recon_metric: 71.3899
Epoch 311/500
1897/1897 [==============================] - 0s 39us/step - loss: 70.7079 - recon_metric: 70.2215 - val_loss: 71.8170 - val_recon_metric: 71.3819
Epoch 312/500
1897/1897 [==============================] - 0s 39us/step - loss: 70.6991 - recon_metric: 70.2109 - val_loss: 71.8132 - val_recon_metric: 71.3769
Epoch 313/500
1897/1897 [=============================

In [16]:
# training #
print(min(history.history['loss']))
# print(min(history.history['recon_metric']))
# validation #
# print(min(history.history['val_loss']))
# print(min(history.history['val_recon_metric']))

70.13056400844029
69.52062
71.43329620361328
70.88510131835938
